In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from difflib import get_close_matches
import textdistance

In [2]:
#bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
corpus = [sent.lower().split() for sent in true]
WORDS = Counter()

In [3]:
for sent in corpus:
    WORDS.update(sent)

In [4]:
WORDS

Counter({'симпатичнейшее': 1,
         'шпионское': 1,
         'устройство': 1,
         'такой': 6,
         'себе': 11,
         'гламурный': 1,
         'фотоаппарат': 2,
         'девушки': 2,
         'бонда': 1,
         'миниатюрная': 1,
         'модель': 2,
         'камеры': 1,
         'superheadz': 1,
         'clap': 1,
         'camera': 1,
         'апофеозом': 1,
         'дня': 3,
         'для': 35,
         'меня': 42,
         'сегодня': 46,
         'стала': 5,
         'фраза': 2,
         'услышанная': 1,
         'в': 292,
         'новостях': 1,
         'поясним': 1,
         'эту': 4,
         'мысль': 2,
         'получатся': 1,
         'вот': 28,
         'такие': 5,
         'язычки': 1,
         'массе': 1,
         'своей': 6,
         'они': 18,
         'конечно': 10,
         'все': 67,
         'очень': 68,
         'милые': 2,
         'насчет': 9,
         'чавеса': 1,
         'разве': 4,
         'что': 146,
         'не': 207,
         'соглаш

In [5]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    return similarities.most_common(1)[0]

In [6]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}
vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [7]:
def get_closest_match_vec(text, X, vec, TOPN=8):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]    
    return [id2word[top] for top in topn]

In [8]:
%%time
get_closest_match_vec('опофеоз', X, vec)

Wall time: 6 ms


['апофеозом',
 'фото',
 'сапфо',
 'фоток',
 'профкома',
 'пофотографировав',
 'фотофайлом',
 'кофе']

In [13]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    topn = get_closest_match_vec(text, X, vec)
    d = {}
    for word in topn: 
        m = metric.normalized_similarity(word, text)
        d[word] = m
    sorted_by_value = sorted(d.items(), key=lambda d: d[1], reverse=True) 
    print(sorted_by_value)
    return sorted_by_value[0][0]

In [14]:
get_closest_hybrid_match('опофеоз', X, vec, metric=textdistance.levenshtein)

[('апофеозом', 0.6666666666666667), ('кофе', 0.4285714285714286), ('фотофайлом', 0.4), ('профкома', 0.375), ('фото', 0.2857142857142857), ('сапфо', 0.2857142857142857), ('фоток', 0.2857142857142857), ('пофотографировав', 0.25)]


'апофеозом'

Ошибки возникают из-за недостаточного объема корпуса:
    корпус сликом мал, в него не помещаются все слова, которые могут существовать

In [15]:
get_closest_hybrid_match('фотография', X, vec, metric=textdistance.levenshtein)

[('фотографии', 0.9), ('топография', 0.8), ('пофотографировав', 0.5625), ('фото', 0.4), ('фотофайлом', 0.4), ('фляги', 0.30000000000000004), ('формат', 0.30000000000000004), ('офиса', 0.19999999999999996)]


'фотографии'

In [16]:
get_closest_hybrid_match('отсутствую', X, vec, metric=textdistance.levenshtein)

[('отсутствие', 0.8), ('чувствую', 0.6), ('советую', 0.4), ('старую', 0.4), ('автомастерскую', 0.3571428571428571), ('сетуют', 0.30000000000000004), ('тусуется', 0.30000000000000004), ('сути', 0.30000000000000004)]


'отсутствие'

In [19]:
get_closest_hybrid_match('отсувтствую', X, vec, metric=textdistance.levenshtein)

[('отсутствие', 0.7272727272727273), ('чувствую', 0.6363636363636364), ('сочувствующие', 0.46153846153846156), ('советую', 0.4545454545454546), ('сетуют', 0.2727272727272727), ('чувствуется', 0.2727272727272727), ('встают', 0.2727272727272727), ('советует', 0.2727272727272727)]


'отсутствие'

N.B. опечатка не помешала найти родственное слово!

In [17]:
get_closest_hybrid_match('профессор', X, vec, metric=textdistance.levenshtein)

[('профессор', 1.0), ('профессий', 0.7777777777777778), ('профессионалов', 0.5714285714285714), ('профессиональном', 0.5), ('профессиональной', 0.5), ('офисе', 0.33333333333333337), ('сапфо', 0.2222222222222222), ('строфы', 0.2222222222222222)]


'профессор'

In [18]:
get_closest_hybrid_match('лежать', X, vec, metric=textdistance.levenshtein)

[('лежат', 0.8333333333333334), ('бежать', 0.8333333333333334), ('пожалеть', 0.5), ('жесть', 0.5), ('желаньем', 0.375), ('житель', 0.16666666666666663), ('жилье', 0.16666666666666663), ('стеллаж', 0.1428571428571429)]


'лежат'